In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import math
from multiprocessing import cpu_count
import warnings
warnings.filterwarnings('ignore')
    

In [2]:
from modules.Utils.utils import loadFromDB

SYMBOL= 'BTC'
df = loadFromDB(f'../backtest_tools/database/database/Binance/1h/{SYMBOL}-USDT.csv')
df.head()

,Open,High,Low,Close,Volume,Timestamp
Date,,,,,,
2020-06-01 02:00:00,9448.27,9508.57,9421.67,9498.31,1742.125917,1590969600
2020-06-01 03:00:00,9498.78,9570.00,9465.30,9551.58,2064.378210,1590973200
2020-06-01 04:00:00,9551.17,9568.61,9526.87,9530.67,1984.956728,1590976800
2020-06-01 05:00:00,9531.70,9571.66,9514.65,9555.79,1620.638978,1590980400
2020-06-01 06:00:00,9556.14,9619.00,9541.96,9549.02,2787.512219,1590984000


In [17]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross


class MLR_w_o_ML(Strategy):
    mlr_window = 21
    sell_t =0.041
    buy_t=-0.011
    
    def computeLaggingLinearRegression(self, df,window=15,)->pd.DataFrame:
        """Compute a lagging moving regression on a column with a window.

        Args:
            df (pd.DataFrame): The dataframe containing features.
            col (str, optional): The column we apply Linear regression on. Defaults to "Close.
            window (int, optional): The window we apply linear regression on. Defaults to 15.

        Returns:
            pd.DataFrame: The entry DataFrame we another column called B_MLR_coefs
        """  
        def computeLinearRegression(x,y)->float:
            """Compute simple linear regression between 2 vectors x and y

            Args:
                x (np.array): x vector
                y (np.array): y vector

            Returns:
                float: The coefficient a corresponding to the linear regression y=ax+b.
            """
            model = LinearRegression(n_jobs=cpu_count()).fit(x,y,)
            return model.coef_[0]
        col="Close"
        df['B_MLR_coefs']= [np.nan]*window+[computeLinearRegression(df.Timestamp.values[i-window:i].reshape(-1, 1), df[col].values[i-window:i]) for i in range(window,len(df))] 
        return df.B_MLR_coefs
    
    def add_threshold(self, arr, threshold):
        return [threshold for _ in range(len(arr))]
      
    def init(self):
        self.mlr = self.I(self.computeLaggingLinearRegression, pd.DataFrame({'Close':self.data.Close,'Timestamp':self.data.Timestamp}), self.mlr_window)
        self.sell_threshold = self.I(self.add_threshold, pd.Series(self.data.Close),self.sell_t)
        self.buy_threshold = self.I(self.add_threshold, pd.Series(self.data.Close),self.buy_t)
        
    def next(self):
        if crossover(self.buy_threshold,self.mlr):
            self.buy()
        elif crossover(self.mlr,self.sell_threshold ):
            self.sell()

bt = Backtest(df, MLR_w_o_ML, cash=10000, commission=.001,exclusive_orders=True)
print(bt.run())
bt.plot()

Start                     2020-06-01 02:00:00
End                       2022-09-23 14:00:00
Duration                    844 days 12:00:00
Exposure Time [%]                   75.285227
Equity Final [$]                  26615.20009
Equity Peak [$]                    68941.3813
Return [%]                         166.152001
Buy & Hold Return [%]               98.375922
Return (Ann.) [%]                     52.6286
Volatility (Ann.) [%]               82.714741
Sharpe Ratio                         0.636266
Sortino Ratio                        1.571151
Calmar Ratio                         0.845452
Max. Drawdown [%]                  -62.249074
Avg. Drawdown [%]                   -2.911804
Max. Drawdown Duration      319 days 10:00:00
Avg. Drawdown Duration        5 days 17:00:00
# Trades                                  203
Win Rate [%]                        45.320197
Best Trade [%]                      26.483503
Worst Trade [%]                    -15.494866
Avg. Trade [%]                    

Row(id='3186', ...)

In [11]:
optim = bt.optimize(mlr_window = range(5,30,2),
            #sell_t=[i/1000 for i in range(1,100,10)],
            #buy_t=[-i/1000 for i in range(1,100,10)],
            maximize='Return [%]',)
optim._strategy

Backtest.optimize:   0%|          | 0/5 [00:00<?, ?it/s]

<Strategy MLR_w_o_ML(mlr_window=21)>

In [12]:
optim

Start                     2020-06-01 02:00:00
End                       2022-09-23 14:00:00
Duration                    844 days 12:00:00
Exposure Time [%]                   75.285227
Equity Final [$]                  26615.20009
Equity Peak [$]                    68941.3813
Return [%]                         166.152001
Buy & Hold Return [%]               98.375922
Return (Ann.) [%]                     52.6286
Volatility (Ann.) [%]               82.714741
Sharpe Ratio                         0.636266
Sortino Ratio                        1.571151
Calmar Ratio                         0.845452
Max. Drawdown [%]                  -62.249074
Avg. Drawdown [%]                   -2.911804
Max. Drawdown Duration      319 days 10:00:00
Avg. Drawdown Duration        5 days 17:00:00
# Trades                                  203
Win Rate [%]                        45.320197
Best Trade [%]                      26.483503
Worst Trade [%]                    -15.494866
Avg. Trade [%]                    

In [68]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross
from ta.volatility import bollinger_hband, bollinger_lband
from ta.trend import ema_indicator

class EMA_cross(Strategy):
    #std_window_l = 1.75
    #std_window_h = 1.75
    window_fast =28
    window_slow=54

    def init(self):        
        self.ema_fast = self.I(ema_indicator, pd.Series(self.data.Close), self.window_fast,)
        self.ema_slow = self.I(ema_indicator, pd.Series(self.data.Close), self.window_slow,)

    def next(self):
        if crossover(self.ema_fast,self.ema_slow):
            self.buy()
        elif crossover(self.ema_slow,self.ema_fast):
            self.sell()

bt = Backtest(df, EMA_cross, cash=10000, commission=.001,exclusive_orders=True)
print(bt.run())
optim = bt.optimize(window_fast = range(5,110,5),
                    window_slow = range(5,110,5),
                    constraint=lambda p: p.window_fast < p.window_slow,
                    maximize='Return [%]',)
print(optim._strategy)
print(optim)

Start                     2020-06-01 02:00:00
End                       2022-09-23 14:00:00
Duration                    844 days 12:00:00
Exposure Time [%]                    1.723712
Equity Final [$]                   8981.76978
Equity Peak [$]                   10077.22354
Return [%]                         -10.182302
Buy & Hold Return [%]               98.375922
Return (Ann.) [%]                   -4.532718
Volatility (Ann.) [%]                3.250087
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.117683
Avg. Drawdown [%]                   -5.889819
Max. Drawdown Duration      840 days 04:00:00
Avg. Drawdown Duration      420 days 11:00:00
# Trades                                    5
Win Rate [%]                              0.0
Best Trade [%]                       -1.33913
Worst Trade [%]                     -4.800089
Avg. Trade [%]                    

Backtest.optimize:   0%|          | 0/5 [00:00<?, ?it/s]

EMA_cross(window_fast=15,window_slow=100)
Start                     2020-06-01 02:00:00
End                       2022-09-23 14:00:00
Duration                    844 days 12:00:00
Exposure Time [%]                    7.996246
Equity Final [$]                  10399.83213
Equity Peak [$]                   12003.53755
Return [%]                           3.998321
Buy & Hold Return [%]               98.375922
Return (Ann.) [%]                    1.707872
Volatility (Ann.) [%]               12.096836
Sharpe Ratio                         0.141183
Sortino Ratio                        0.258563
Calmar Ratio                         0.125501
Max. Drawdown [%]                   -13.60845
Avg. Drawdown [%]                   -2.582699
Max. Drawdown Duration      782 days 09:00:00
Avg. Drawdown Duration       52 days 11:00:00
# Trades                                   20
Win Rate [%]                             20.0
Best Trade [%]                      20.558896
Worst Trade [%]                     -2

In [3]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross
from ta.volatility import bollinger_hband, bollinger_lband

class Bollinger_start(Strategy):
    std_window_l = 1.5
    std_window_h = 1.5
    window_h =20
    window_l=20

    def init(self):
        
        self.lband = self.I(bollinger_lband, pd.Series(self.data.Close), self.window_l,self.std_window_l )
        self.hband = self.I(bollinger_hband, pd.Series(self.data.Close), self.window_h,self.std_window_h )
        
    def next(self):
        if crossover(self.lband,self.data.Close):
            self.buy()
        elif crossover(self.data.Close,self.hband):
            self.sell()

bt = Backtest(df, Bollinger_start, cash=10000, commission=.001,exclusive_orders=True)
print(bt.run())
optim = bt.optimize(window_h = range(5,50,5),
                    window_l = range(5,50,5),
                    std_window_l=[i/10 for i in range(10,20,5)],
                    std_window_h=[i/10 for i in range(10,20,5)],
                    maximize='Return [%]',)
print(optim._strategy)
print(optim)

Start                     2020-06-01 02:00:00
End                       2022-09-23 14:00:00
Duration                    844 days 12:00:00
Exposure Time [%]                    6.479972
Equity Final [$]                   9756.23359
Equity Peak [$]                   10895.91283
Return [%]                          -2.437664
Buy & Hold Return [%]               98.375922
Return (Ann.) [%]                    -1.09165
Volatility (Ann.) [%]                5.899886
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -10.94685
Avg. Drawdown [%]                   -1.786787
Max. Drawdown Duration      836 days 03:00:00
Avg. Drawdown Duration       60 days 06:00:00
# Trades                                  118
Win Rate [%]                        45.762712
Best Trade [%]                       2.147126
Worst Trade [%]                     -4.433817
Avg. Trade [%]                    

Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

Bollinger_start(window_h=40,window_l=35,std_window_l=1.0,std_window_h=1.5)
Start                     2020-06-01 02:00:00
End                       2022-09-23 14:00:00
Duration                    844 days 12:00:00
Exposure Time [%]                    6.420704
Equity Final [$]                   9935.38717
Equity Peak [$]                   10963.18245
Return [%]                          -0.646128
Buy & Hold Return [%]               98.375922
Return (Ann.) [%]                   -0.279611
Volatility (Ann.) [%]                5.024508
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -9.659105
Avg. Drawdown [%]                   -1.197147
Max. Drawdown Duration      800 days 16:00:00
Avg. Drawdown Duration       29 days 01:00:00
# Trades                                   89
Win Rate [%]                        41.573034
Best Trade [%]                       2.613682
Worst